In [1]:
import sys
sys.path.append('../src')
sys.path.append('../results')
import json
import pandas

In [2]:
import pandas as pd
import os
import re

base_dir = '../results/score_model/'

# Regex pattern for matching file names
pattern = r'gpt2_\d+_evaluate_mechanism.csv'

# Initialize an empty list to store file paths
file_paths = []

# Walk through the directory
for dirname, _, filenames in os.walk(base_dir):
    # Filter and append the files that match the pattern
    file_paths.extend([os.path.join(dirname, filename) for filename in filenames if re.match(pattern, filename)])



# Initialize a DataFrame to store aggregated data
aggregated_data = pd.DataFrame()

# Loop over each file
for file in file_paths:
    # Read the current CSV file
    current_data = pd.read_csv(file)

    # If the aggregated_data DataFrame is empty, initialize it with the structure of the current_data
    if aggregated_data.empty:
        aggregated_data = current_data.copy()
        # Initialize columns for mean and standard deviation
        for column in ["target_true", "target_false", "other"]:
            aggregated_data[column + "_mean"] = 0
            aggregated_data[column + "_std"] = 0
    else:
        # For each column, calculate cumulative sum and square sum for standard deviation calculation
        for column in ["target_true", "target_false", "other"]:
            aggregated_data[column + "_mean"] += current_data[column]
            aggregated_data[column + "_std"] += current_data[column]**2

# Number of files
num_files = len(file_paths)

# Calculate the mean and standard deviation
for column in ["target_true", "target_false", "other"]:
    aggregated_data[column + "_mean"] /= num_files
    aggregated_data[column + "_std"] = (aggregated_data[column + "_std"] / num_files - aggregated_data[column + "_mean"]**2).apply(lambda x: x**0.5)

# Drop the original columns and keep only mean and std columns
aggregated_data.drop(columns=["target_true", "target_false", "other"], inplace=True)





In [3]:
aggregated_data

,model_name,orthogonalize,target_true_mean,target_true_std,target_false_mean,target_false_std,other_mean,other_std
0,gpt2,True,484.87,53.140692,8867.73,891.731987,547.40,59.100085
1,gpt2-medium,True,820.94,86.762068,8333.54,838.392992,745.52,79.306933
2,gpt2-large,True,1096.42,113.592005,7885.51,793.288970,918.07,95.289585
3,gpt2-xl,True,2260.58,230.114327,6549.26,659.798524,1090.16,113.496848


In [4]:
# drop the row with orthogonalize True
gpt2_clean = pd.read_csv("../results/gpt2_evaluate_mechanism.csv")
gpt2_clean = gpt2_clean[gpt2_clean['orthogonalize'] == False]

In [5]:
aggregated_data =aggregated_data.rename(columns={"target_true_mean": "target_true", "target_false_mean": "target_false", "other_mean":"other"})

In [6]:

# Adding 0 for the standard deviation in the first dataset
for column in ["target_true", "target_false", "other"]:
    gpt2_clean[column + "_std"] = 0

# Concatenate the two datasets
final_dataset = pd.concat([gpt2_clean, aggregated_data], ignore_index=True)

# Save the final dataset
final_dataset.to_csv("../results/plot_data/gpt2_count.csv", index=False)

In [8]:
final_dataset.to_csv("../results/plot_data/gpt2_count.csv", index=False)